<a href="https://colab.research.google.com/github/chowell2000/Advent_of_Code/blob/master/Picard_lstm_tests_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# import numpy as np
# import random
import json
import os
import pandas as pd
import sys
import urllib.request

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('drive/My Drive/s01e01.json') as f:
  farpoint_json = f.read()

In [ ]:
# farpoint_json = open('drive/My Drive/s01e01.json')

# open()

In [ ]:
print(farpoint_json[0:200])

{
  "airdate": 559800000,
  "scenes": [
    {
      "dialogue": [
        {
          "character": "PICARD",
          "line": "Captain's log, stardate 41153.7. Our destination is planet Deneb Four, b


In [ ]:
farpoint = json.loads(farpoint_json)

In [ ]:
keys = farpoint.keys()
print(keys)

dict_keys(['airdate', 'scenes', 'schedule', 'stardate', 'title'])


In [ ]:
# print(farpoint['scenes'][1])
scenes = farpoint['scenes']
len(scenes)
scenes[1]['dialogue'][0]

{'character': 'PICARD',
 'line': "You will agree, Data, that Starfleet's orders are difficult?"}

In [ ]:
lines = {}
for n in range(len(scenes)):
  for i in range(len(scenes[n]['dialogue'])):
    # print(scenes[1]['dialogue'][i]['line'])
    lines[scenes[n]['dialogue'][i]['line']] = scenes[n]['dialogue'][i]['character']

In [ ]:
lines

{'Ah! To seek covertly, to go stealthily, to slink, slither': 'DATA',
 'As simple as that.': 'PICARD',
 "Captain's log, supplementary. The frozen form of Lieutenant Torres has been rushed to sickbay. The question now is the incredible power of the Q being. Do we dare oppose it?": 'PICARD',
 "Captain, I'm sensing a powerful mind.": 'TROI',
 'Controls to full stop, sir. Now reading full stop, sir.': 'TORRES',
 'Data, call medics.': 'PICARD',
 'Data, how can you be programmed as a virtual encyclopedia of human information without knowing a simple word like snoop?': 'PICARD',
 'Difficult? Simply solve the mystery of Farpoint Station.': 'DATA',
 'Exactly, yes.': 'PICARD',
 'Farpoint Station. Even the name sounds mysterious.': 'TROI',
 'Glide, creep, skulk, pussyfoot, gumshoe.': 'DATA',
 'He would not have injured you. Do you recognize this, the stun setting?': 'PICARD',
 "He's frozen.": 'TROI',
 'I present myself to thee as a fellow ship captain, that thou mayst better understand me. Go bac

In [ ]:
tng_lines = {}
basepath = 'drive/My Drive/tng/'
for filename in os.listdir(basepath):
      # print(os.path.join("./DataFiles", filename))
      
      with open(basepath + filename) as f:
        script_json = f.read()
        script = json.loads(script_json)

        for n in range(len(script['scenes'])):
          for i in range(len(script['scenes'][n]['dialogue'])):
          # print(scenes[1]['dialogue'][i]['line'])
            tng_lines[script['scenes'][n]['dialogue'][i]['line']] = script['scenes'][n]['dialogue'][i]['character']

In [ ]:
line_list = tng_lines.to_list()
picard_list = [line[1] == 'PICARD' for line in line_list]


In [ ]:
picard = [k for k, v in tng_lines.items() if v == 'PICARD']

In [ ]:
with open('drive/My Drive/picard_lines.json', 'w') as f:
  json.dump(picard, f)


In [3]:
with open('drive/My Drive/picard_lines.json', 'r') as f:
  picardtest = json.loads(f.read())


In [4]:
picardtest[-20:-1]

['All hands, brace for impact!',
 'Maintain course and speed. Mister Data, how long until we reach the center?',
 'Initiate warp shell!',
 'No, we have to maintain the static warp shell as long as possible!',
 'Transfer emergency power to the antimatter containment system!',
 'Maintain position! Mister La',
 'Data, report!',
 'Are you saying that it worked? We collapsed the anomaly?',
 'Did it work?',
 'What about my crew?',
 'You had a hand in helping me get out of this.',
 'I sincerely hope that this is the last time that I find myself here.',
 'When I realized the paradox.',
 "Q, what is it that you're trying to tell me?",
 "Mister Worf, what's the date?",
 "No. No. No. I think I'll get back to bed. I could use some sleep.",
 "Captain's log, supplemental. Starfleet Command reports no unusual activity along the Neutral Zone, and there is no sign of a temporal anomaly. It would appear that I am the only member of the crew to retain any knowledge of the events I experienced.",
 "No. I,

In [5]:
len(picardtest)

11132

In [6]:
line_len = [len(l) for l in picardtest]

In [7]:
max(line_len)

1080

In [8]:
longlines = [l for l in picardtest if len(l) > 282]
longlogs = [l for l in picardtest if l[0:7] == 'Captain' and len(l) > 280]
len(longlogs)

64

In [9]:
logs = [l for l in picardtest if l[0:7] == 'Captain' and len(l) < 283]
len(logs)

343

In [10]:
len_logs = [len(l) for l in logs]

In [11]:
sum(len_logs) + 58*280

77159

In [12]:
lines_280 = [l for l in picardtest if len(l)<281]
lines_280_len = [len(l) for l in lines_280]

In [13]:
charcount = pd.DataFrame(data=lines_280_len)
charcount.describe()

,0
count,10865.000000
mean,70.226783
std,55.205666
min,1.000000
25%,30.000000
50%,53.000000
75%,94.000000
max,280.000000


In [14]:
full_text_no_padd = ' '.join(lines_280)

In [16]:
lines_280 = ['{:290}'.format(l) for l in lines_280]

In [17]:

import numpy as np
import pickle
import random
import sys
import os
import tensorflow as tf
import urllib.request

from __future__ import print_function

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop, Nadam

In [20]:
full_text = ' '.join(lines_280)

In [21]:
character_encoding = dict(zip(sorted(set(full_text)), range(len(set(full_text)))))
character_decoding = dict(zip(range(len(set(full_text))), sorted(set(full_text))))

In [62]:
# Create the sequence data

maxlen = 40
step = 1

encoded_280 = []

for line in lines_280:
  encoded = [character_encoding[c] for c in line]
  encoded_280.append(encoded)

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for encoded in encoded_280:
  for i in range(0, len(encoded) - maxlen, step):
      
      sequences.append(encoded[i : i + maxlen])
      next_char.append(encoded[i + maxlen])
      if encoded[i + 38 :i + maxlen] == [0]*2:
        break

print('sequences: ', len(sequences))



sequences:  416136


In [25]:
character_encoding[' ']

0

In [63]:
# Create x & y

x = np.zeros((len(sequences), maxlen, len(character_encoding)), dtype=np.bool)
y = np.zeros((len(sequences),len(character_encoding)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1
        

In [25]:
# np.save('drive/My Drive/x_shake_2.npy', x)
# np.save('drive/My Drive/y_shake_2.npy', y)

In [50]:
def sample(preds, temp = .8):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [52]:
def on_epoch_end_3(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    # start_index = random.randint(0, len(full_text_no_padd) - maxlen - 1)
    start_index = random.randint(0, len(lines_280))
    
    generated = ''
    
    # sentence = full_text_no_padd[start_index: start_index + maxlen]
    sentence = lines_280[start_index][0:40]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '" /n')
    sys.stdout.write(generated)
    
    space_check = ''
    for i in range(290):
        x_pred = np.zeros((1, maxlen, len(character_encoding)))
        for t, char in enumerate(sentence):
            x_pred[0, t, character_encoding[char]] = 1
            
        preds = model3.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = character_decoding[next_index]
        
        sentence = sentence[1:] + next_char

        space_check = space_check + next_char
        if space_check[-5:] == '     ':
          break


        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


    generated = ''
    
    sentence = full_text_no_padd[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: ' + sentence + ' /n')
    sys.stdout.write(generated)
    
    for i in range(290):
        x_pred = np.zeros((1, maxlen, len(character_decoding)))
        for t, char in enumerate(sentence):
            x_pred[0, t, character_encoding[char]] = 1
            
        preds = model3.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temp= .5)
        next_char = character_decoding[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()



print_callback_1 = LambdaCallback(on_epoch_end=on_epoch_end_3)

In [65]:
file_path = "drive/My Drive/picard_lstm/picard_model_3-{epoch:03d}-{loss:.4f}"

save_callback = ModelCheckpoint(filepath = file_path,
                                verbose=1, save_best_only=False,
                                save_weights_only=False,  ) 

weights_path = "drive/My Drive/picard_lstm/picard_weights_3-{epoch:03d}-{loss:.4f}"

save_weights_callback = ModelCheckpoint(filepath = weights_path,
                                verbose=1, save_best_only=False,
                                save_weights_only=True) 

In [66]:
# build the model: a three LSTM model

model3 = Sequential()
model3.add(LSTM(128, input_shape=(maxlen, len(character_encoding)), return_sequences=True,
                dropout=.2))
model3.add(LSTM(256, return_sequences=True, dropout=.2))
model3.add(LSTM(128, dropout=.2, recurrent_dropout=.2 ))
model3.add(Dense(len(character_encoding), activation='softmax'))

optimizer = Nadam(learning_rate=0.001, )

model3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'] )

In [67]:
model3.load_weights("drive/My Drive/picard_lstm/picard_model_2-113-0.9608")

In [ ]:
# fit the model

model3.fit(x, y,
          batch_size=32,
          epochs=200,
          # callbacks=[save_callback, print_callback_1, save_weights_callback])
          callbacks=[ print_callback_1, save_callback])
# , validation_split = .2

Epoch 1/200
1062/1062 [==============================] - 192s 170ms/step - loss: 2.6420 - accuracy: 0.2766

----- Generating text after Epoch: 0
----- Generating with seed: "he same respect as any other being.Did e" /n
he same respect as any other being.Did er nes ars miu mein pul aices olllh  him S wesses.    
----- Generating with seed: he same respect as any other being.Did e /n
he same respect as any other being.Did eitir sote care er sev ar anniu ti iasir tor tor ses toe mise the  hor in tiu ler as ess bes.                                                                                                                                                                                                      

Epoch 00001: saving model to drive/My Drive/picard_lstm/picard_model_2-001-2.6420


INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-001-2.6420/assets


INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-001-2.6420/assets


Epoch 2/200
1062/1062 [==============================] - 181s 170ms/step - loss: 2.2678 - accuracy: 0.3629

----- Generating text after Epoch: 1
----- Generating with seed: "d of any changes, no matter how small.Ca" /n
d of any changes, no matter how small.Cand ou'de wastifren hasunitarg ohicustont cantrind ho mie Bonns it ik toe    
----- Generating with seed: d of any changes, no matter how small.Ca /n
d of any changes, no matter how small.Cantertinsing the t                                                                                                                                                                                                                                                                                 

Epoch 00002: saving model to drive/My Drive/picard_lstm/picard_model_2-002-2.2678


INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-002-2.2678/assets


...

INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-112-0.9702/assets


Epoch 113/200
1062/1062 [==============================] - 179s 168ms/step - loss: 0.9608 - accuracy: 0.7027

----- Generating text after Epoch: 112
----- Generating with seed: "th and how Doctor Crusher brought her ba" /n
th and how Doctor Crusher brought her bare teree, Lieutenant Worf, anything. NoA! you stonl to you, Mister Data. Somf waip. I mave a structure toune ship. Captain Dirasting. Cound lact indicated, and not going oor welcoment? The Saldsion of the sumpoctover, Jorel and waip of plering in made through time. There's a missions light
----- Generating with seed: th and how Doctor Crusher brought her ba /n
th and how Doctor Crusher brought her bast interplanetary code, you must be bock. The sarpase. Let's set a moment. Colonist I don't think best the donet convernative that the survivong through the perimetely could actives are planuter of the Neutral Zone, allow that you're still to save the courne sefension. You are resolutely r

Epoch 00113: saving model to drive/My Driv

INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-113-0.9608/assets


INFO:tensorflow:Assets written to: drive/My Drive/picard_lstm/picard_model_2-113-0.9608/assets


Epoch 114/200
1062/1062 [==============================] - 180s 170ms/step - loss: 0.9716 - accuracy: 0.7017

----- Generating text after Epoch: 113
----- Generating with seed: "ed Kataan.No, I'm not surprised. Its sun" /n
ed Kataan.No, I'm not surprised. Its sunsed by shouldr without kick that. The poterterete troi  The Remulan offecer transporter room, Lieutenant. I hove ever year. I shouldn't way it's the talking forward to the rupture to tell the two diseosed without syin. I've been attempt asd, Avay where it's a mine the Federation Starship, 
----- Generating with seed: ed Kataan.No, I'm not surprised. Its sun /n
ed Kataan.No, I'm not surprised. Its sunsored the shuttle raft. He's onderstand, you would h

In [ ]:
model3 = tf.keras.models.load_model('drive/My Drive/picard_lstm/picard_model_1-064-1.0960')

In [32]:
seed = "Captain's Log, Stardate 97214 we are on"

print(len(seed))

seed_encoded = [character_encoding[c] for c in seed]

print(type(seed_encoded))

seed_encoded







29
<class 'list'>


[24,
 48,
 63,
 67,
 48,
 56,
 61,
 2,
 66,
 0,
 33,
 62,
 54,
 6,
 0,
 40,
 67,
 48,
 65,
 51,
 48,
 67,
 52,
 0,
 18,
 16,
 11,
 10,
 13]

In [ ]:
def generate_picard():
  sentence

In [ ]:
# fit the model

model3.fit(x, y,
          batch_size=32,
          epochs=100,
          callbacks=[save_callback, print_callback_1, save_weights_callback])
          # callbacks=[ print_callback_1])
# , validation_split = .2

Epoch 1/100
 9866/13005 [=====================>........] - ETA: 16:48 - loss: 1.7760 - accuracy: 0.4897

In [ ]:
# fit the model

model3.fit(x, y,
          batch_size=32,
          epochs=100,
          callbacks=[save_callback, print_callback_1, save_weights_callback])
          # callbacks=[ print_callback_1])
# , validation_split = .2

In [ ]:
# start_index = random.randint(0, len(full_text) - maxlen - 1)
    
generated = ''

# sentence = full_text[start_index: start_index + maxlen]

sentence = "Stardate"

generated += sentence
    
print('----- Generating with seed: "' + sentence + '" /n')
sys.stdout.write(generated)
    
space_check = ''
for i in range(290):
        x_pred = np.zeros((1, maxlen, len(character_encoding)))
        for t, char in enumerate(sentence):
            x_pred[0, t, character_encoding[char]] = 1
            
        preds = model3.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temp = 0.6)
        next_char = character_decoding[next_index]
        
        sentence = sentence[1:] + next_char

        space_check = space_check + next_char
        if space_check[-5:] == '     ':
          break


        
        sys.stdout.write(next_char)
        sys.stdout.flush()
print()